# DeepSpeed Inference: Enabling Efficient Inference of Transformer Models at Unprecedented Scale ([ссылка](https://arxiv.org/pdf/2207.00032))

**DeepSpeed Inference**, комплексное системное решение для вывода моделей трансформаторов для решения вышеупомянутых проблем. DeepSpeed Inference состоит из 
 1. решения для вывода с несколькими графическими процессорами, которое сводит к минимуму задержку и максимизирует пропускную способность моделей как плотных, так и разреженных трансформаторов, когда они помещаются в совокупную память графического процессора
 2. решения для гетерогенного вывода, которое использует память CPU и NVMe в дополнение к памяти GPU и вычислительным ресурсам для обеспечения высокой пропускной способности вывода с большими моделями, которые не помещаются в совокупную память GPU.

## Inference-optimized transformer kernels

**Пробелема:** 

Производительность малых пакетов ограничена использованием пропускной способности памяти при чтении весовых коэффициентов модели. Существуют три основные проблемы, связанные с оптимизацией пропускной способности памяти при выводе небольших пакетов.
 1. из-за ограниченной работы в различных ядрах, выполняющих операции трансформаторного слоя с использованием небольшого пакета, производительность вывода страдает от накладных расходов на вызов ядра.
 2. каждый вызов ядра записывает данные в глобальную память, которая считывается ядрами графического процессора во время следующего вызова ядра, и эта передача данных между ядрами графического процессора и глобальной памятью добавляет дополнительные накладные расходы. 
 3. нет библиотек, эффективно работающих с малыми пакетами. С другой стороны, производительность вывода больших пакетов ограничена использованием вычислительных ресурсов, и хотя тяжелые вычислительные операции, такие как GeMM внутри трансформаторного слоя, могут обеспечить очень хорошее использование вычислений с использованием библиотек CUBLAS и CUTLASS, общее использование все еще может быть ограничено накладными расходами на запуск ядра и передачей данных между ядрами графического процессора и глобальной памятью между различными ядрами, отличными от GeMM.

### Deep-fusion

трансформеры состоят из операторов, таких как преобразования структуры данных (data layout transformations), редукции (reductions) и операции матричного умножения (GeMMs), которые создают зависимости по данным между блоками потоков (thread blocks), что делает их сложными для слияния. Это связано с тем, что на GPU, если данные, созданные одним блоком потоков, используются другим блоком, требуется глобальная синхронизация памяти, что приводит к запуску нового ядра (kernel).

Чтобы избежать необходимости глобальной синхронизации, Deep-Fusion разбивает пространство вычислений (computation-space) на тайлы (tiles) вдоль измерений итерационного пространства (iteration space), которые не создают зависимостей по данным между тайлами, и выполняет их параллельно в разных блоках потоков. Измерения пространства вычислений, которые содержат зависимости по данным, не разбиваются на тайлы, а обрабатываются одним и тем же блоком потоков. После такого разбиения два оператора могут быть объединены с помощью Deep-Fusion, если каждый тайл второго оператора зависит ровно от одного выходного тайла первого оператора. Выполняя слияние на уровне тайлов, Deep-Fusion может объединять не только поэлементные операции, но и редукции, транспонирование данных и операции GeMMs, при условии отсутствия зависимостей между тайлами.


### Cooperative-Group Reduction

Наша специализированная реализация GeMM спроектирована таким образом, чтобы ее можно было объединить с Deep-Fusion при максимальном использовании пропускной способности памяти. Его дизайн можно рассматривать в трех частях:
 1) **Стратегии разбиения на листы:** На рисунке ниже (а) показано наше планирование GeMM для умножения тонкой матрицы. Сначала мы расположим вычисления вдоль выходной размерности. Это позволяет нам реализовать GeMM с использованием одного ядра, сохраняя редукцию в пределах тайла. Для небольших моделей, где выходная размерность слишком мала для создания достаточного количества параллельных листов для достижения хорошей пропускной способности памяти, мы также выделяем входную размерность и реализуем GeMM в виде двух ядер, чтобы обеспечить сокращение между листами.

 2) **Сокращение кооперативной группы:** При вышеупомянутой стратегии разбиения на листы, каждый warp в thread block отвечает за получение частично уменьшенного результата для плитки output, и окончательное сокращение необходимо для всех warp внутри thread block. Обычно это реализуется в виде сокращения разделяемой памяти на основе двоичного дерева, которое требует нескольких синхронизаций на уровне warp, тем самым создавая узкое место производительности. Чтобы избежать этого, мы выполняем транспонирование одного макета данных в общей памяти таким образом, чтобы частичные результаты одного и того же выходного элемента были непрерывными в памяти и могли быть уменьшены за счет одного варпа с использованием коллективов кооперативных групп непосредственно в регистрах. В конце концов, первый поток каждой основы содержит конечный результат и записывает его в общую память. Результаты в общей памяти являются непрерывными, что позволяет осуществлять объединенную запись в глобальную память.

![GeMM-schreduling](../Pictures/DeepSpeed-inference/GeMM-schreduling.png)

**Пояснение:**

$$
(X_1 X_2 X_3 X_4) *  
\begin{pmatrix}\
    Y_1^{(1)} &  Y_2^{(1)} & Y_3^{(1)} & Y_4^{(1)} \\ 
    Y_1^{(2)} &  Y_2^{(1)} & Y_3^{(2)} & Y_4^{(2)} \\
    Y_1^{(3)} &  Y_2^{(3)} & Y_3^{(3)} & Y_4^{(3)} \\ 
    Y_1^{(4)} &  Y_2^{(4)} & Y_3^{(4)} & Y_4^{(4)} \\
\end{pmatrix} + 

\begin{pmatrix}\
    B_1^{(1)} & B_2^{(1)} & B_3^{(1)} & B_4^{(1)} \\ 
    B_1^{(2)} & B_2^{(1)} & B_3^{(2)} & B_4^{(2)} \\
    B_1^{(3)} & B_2^{(3)} & B_3^{(3)} & B_4^{(3)} \\ 
    B_1^{(4)} & B_2^{(4)} & B_3^{(4)} & B_4^{(4)} \\
\end{pmatrix} =>

\begin{pmatrix}\
    X_1Y_1^{(1)} &  X_1Y_2^{(1)} & X_1Y_3^{(1)} & X_1Y_4^{(1)} \\ 
    X_2Y_1^{(2)} &  X_2Y_2^{(1)} & X_2Y_3^{(2)} & X_2Y_4^{(2)} \\
    X_3Y_1^{(3)} &  X_3Y_2^{(3)} & X_3Y_3^{(3)} & X_3Y_4^{(3)} \\ 
    X_4Y_1^{(4)} &  X_4Y_2^{(4)} & X_4Y_3^{(4)} & X_4Y_4^{(4)} \\
\end{pmatrix} + 

\begin{pmatrix}\
    B_1^{(1)} & B_2^{(1)} & B_3^{(1)} & B_4^{(1)} \\ 
    B_1^{(2)} & B_2^{(1)} & B_3^{(2)} & B_4^{(2)} \\
    B_1^{(3)} & B_2^{(3)} & B_3^{(3)} & B_4^{(3)} \\ 
    B_1^{(4)} & B_2^{(4)} & B_3^{(4)} & B_4^{(4)} \\
\end{pmatrix} \\

=> (Transpose + Reduce) =>

\begin{pmatrix}\
    \sum\limits_{k=1}^{4} X_kY_i^{(k)} + B_i^{(k)}
\end{pmatrix}_{i=1,2,3,4 (index \ GPU)}



$$
Затем снова делаем Transpose, чтобы вернуться к исходному состоянию

 3) **Использование полной строки кэша:** В архитектуре GPU каждая строка кэша $L_1$ имеет размер 128 байт, однако объединенный доступ к памяти с одним элементом FP16 или INT8 на поток в warp не может полностью использовать всю строку кэша. Чтение нескольких элементов в потоке вдоль выходного измерения для решения этой проблемы уменьшает количество параллельных листов, что также снижает пропускную способность памяти. Таким образом, наше решение состоит в том, чтобы транспонировать матрицу весов во время инициализации таким образом, чтобы $M$ строк для каждого столбца были непрерывными в памяти, что позволило бы каждому потоку прочитать $M$ элементов вдоль входного измерения (рисунок ниже). Мы устанавливаем $M$ как 2 для половинной точности и 4 для типов данных INT8 с учетом 128-байтовой строки кэша.

![Cache-optimization](../Pictures/DeepSpeed-inference/Cache-optimization.png)

### Собираем вместе

![Putting-small-transformer-kernel](../Pictures/DeepSpeed-inference/Putting-small-transformer-kernel.png)

**Small-batch Transformer Kernel:** На рисунке выше показаны различные компоненты трансформаторного слоя и операции, которые рассматриваются для Deep-Fusion в случае вывода небольших партий. Как показано на рисунке, мы объединяем операции внутри трансформаторного слоя в четырех основных областях:
 1) QKV GeMM и входной слой-норма
 2) транспозиция плюс внимание
 3) пост-внимание-норма и промежуточный GeMM
 4) смещение и остаточное добавление.
 
Для поддержки слияния GeMM с остальными операциями в одном ядре для (3) мы транслируем входной пакет через SM и выполняем те же операции, которые поступают до GeMM, так что нет необходимости обмениваться данными между SM для добавления расписания GeMM. Мы видим, что, несмотря на репликацию работы между SM, мы все еще получаем выигрыш в производительности по сравнению с нереплицируемой реализацией ядра без слияния для очень маленьких размеров пакетов. **(разобраться, как работает Deep-fusion)**

**Large-batch Transfomer Kernel:** Мы следуем той же стратегии слияния, что и обсуждалась выше, с той разницей, что мы используем CUBLAS для операций GeMM и оставляем их необъединенными.

**Support for Different Data Types:** Наши ядра поддерживают типы данных FP32, FP16 и INT8 для операций GeMM. Для поддержки INT-8 мы используем реализацию CUTLASS INT8 GeMM, настроенную на различные размеры партий. Мы также добавляем операцию квантования перед GeMM, который мы объединяем с помощью DeepFusion, и деквантование после GeMM, который мы объединяем с помощью функции эпилога CUTLASS

**Eliminating Kernel Invocation Overhead via Cuda-Graph:** Для моделей малого и среднего размера с небольшими размерами пакетов, по мере сокращения фактического времени выполнения ядер, основное узкое место задержки смещается с выполнения ядра на нагрузку на запуск ядра на стороне процессора. Чтобы решить эту проблему, мы добавляем поддержку CUDA-Graph в наш конвейер вывода. В частности, мы сохраняем трассировку ядер при их первом запуске во время прямых вычислений при выводе и создаем вычислительный граф, который можно повторно использовать для следующих запросов, что в значительной степени устраняет накладные расходы на запуск ядра и значительно повышает производительность

**Замечание:** Посмотреть [CUTLASS](https://developer.nvidia.com/blog/cutlass-linear-algebra-cuda/) и [CUDA-Graph](https://developer.nvidia.com/blog/cuda-graphs/)

### Inference-adapted dense transformer models on many GPU systems

**Проблема:** необходимо решить 2 задачи, связанные с несколькими GPU

 1. дальнейшее снижение задержки за счет использования совокупной пропускной способности памяти между графическими процессорами 
 2. увеличение емкости памяти за счет аггрегации памяти GPU на нескольких нодах для размещения большой модели

## Aggregate Memory Bandwidth via Tensor Parallelism:

По сути реализация страндартная. Может делать TP автоматически.

**Проблема:** TP не может быть эффективно масштабировано за пределы одного узла из-за значительных затрат на обмен данными. Таким образом, для дальнейшего масштабирования до многоузловых систем DeepSpeed Inference использует конвейерный параллелизм.

![PP-And-TP](../Pictures/DeepSpeed-inference/Pipeline-And-Tensor-parallelisms.png)

## Aggregate Memory via Pipeline Parallelism

### Проблемы:

Применение PP в выводе является нетривиальным процессом и требует других соображений, чем обучение. 

Во-первых, декодеры трансформеров являются авторегрессионными, т.е. входными данными для вывода модели являются ранее сгенерированные выходы. Таким образом, при создании последовательности следующий токен в последовательности является функцией предыдущих токенов. Существующие конвейеры обучения делают вывод на основе гранулярности пакетов, поэтому границы пакетов определяются зависимостями данных при создании последовательностей. Эти зависимости данных вызывают частые пузырьки конвейера, которые ухудшают производительность вывода (рисунок ниже).

![pipeline-problem](../Pictures/DeepSpeed-inference/pipeline-problem.png)

Во-вторых, модели авторегрессионной генерации имеют две четко выраженные фазы:
 1) фаза обработки запроса, когда весь запрос на ввод обрабатывается для создания первого токена
 2) фаза генерации токена, где результаты обработки запроса повторно используются с помощью KV-кэширования, и новое вычисление зависит только от одного ранее сгенерированного токена. Поскольку количество токенов, обрабатываемых на каждом этапе, кардинально отличается, они имеют разные характеристики производительности, требующие разных соображений.

### Оптимизации:

**Hiding data dependencies and hybrid scheduling**

Предположим, что наша цель состоит в том, чтобы вывести пакет последовательностей $B$, $s_1, s_2, ..., s_B$. Мы разделяем последовательности на группы микропакетов, где один микропакет является единицей вычислений, предоставляемой для каждого вызова ядра. Микропакет проходит через этапы конвейера модели до тех пор, пока на последнем этапе модели не будут созданы следующие токены. Если последовательность $s_i$ не завершается, сгенерированный токен будет использоваться в качестве входных данных для генерации следующего токена. На рисунке выше показан наш график генерации параллельных последовательностей pipeline. Задаем количество микропартий на глубину pipeline, $P$. Наличие по крайней мере $P$ микропакетов имеет решающее значение для использования всех этапов конвейера, но необходимо избегать дополнительных микропакетов из-за задержек и затрат памяти при большем размере пакета. Тем не менее, нельзя многократно выводить партию микропартий $P$ без значительного снижения эффективности из-за пузырей конвейера. Мы избегаем промежуточных пузырей конвейера, динамически ставя в очередь микропакеты сгенерированных токенов до тех пор, пока последовательности не прекратятся. Полученный график амортизирует пузырь конвейера по всем сгенерированным токенам без выделения дополнительных активаций из большего размера партии. (рисунок выше, b)

Однако этого недостаточно для получения наилучшей производительности. Авторегрессионные модели, такие как GPT-3, часто состоят из двух стадий, которые имеют разные характеристики производительности, и использование одного и того же размера микропартии для обеих стадий является неоптимальным.

Promt processing имеет большое количество токенов на семпл, которые могут загрузить вычислительные ресурсы графического процессора, а выбор микропакетов влияет только на пузырь конвейера, но не на время выполнения графического процессора. Однако для генерации токена каждый семпл имеет один токен, общее количество токенов во всем микропакете невелико, а время выполнения ядра полностью ограничено пропускной способностью памяти. Это означает, что время выполнения микропартии не сильно меняется с изменением размера микропартии, так как большая часть времени тратится на выборку параметров модели. Однако общее время выполнения пропорционально количеству микропакетов, так как прямой проход для каждого микропакета требует повторного извлечения весов. Таким образом, для эффективной генерации токенов необходимо свести к минимуму количество микропартий, сохраняя при этом достаточно большое, чтобы скрыть пузырь конвейера  

Чтобы удовлетворить различные требования к быстрой обработке и генерации токенов, мы применяем гибридную стратегию планирования, в которой мы используем разное количество микропакетов для быстрой обработки и генерации токенов. На рисунке ниже показано, как работает гибридное планирование. Мы используем большее количество микропакетов на этапе оперативной обработки, чтобы свести к минимуму пузырь конвейера, в то время как на этапе генерации токенов мы уменьшаем количество микропакетов, чтобы сократить общее время выполнения.

![hybrid-micro-batch-count](../Pictures/DeepSpeed-inference/hybrid-micro-batch-count.png)

**Пояснение:** На последнем конвеере мы просто объеденили 4 микробатча в 2

**Offloading Activations to CPU Memory**

Кэшированные тензоры активации key и value имеют предсказуемый шаблон повторного использования. Активации последовательности $s_i$ не будут использоваться снова до тех пор, пока не будет сгенерирован следующий токен $s_i$. Когда выделенная память активации превышает пороговое значение, мы перегружаем некоторые активации с графического процессора на память процессора, пока они не используются. Сэкономленная память графического процессора позволяет использовать партии большего размера и обеспечивает более эффективное использование системы.

**Communication optimization**

Производительность инференса в конечном итоге снизится, если ядра трансформаторов будут останавливаться на связи для разгрузки памяти процессора через PCIe с низкой пропускной способностью. Чтобы избежать задержек, мы перекрываем коммуникацию с вычислениями и, что более важно, используем оптимизированную для архитектуры стратегию разгрузки. Большинство системных архитектур не имеют уникальной шины PCIe для каждого графического процессора и используют один канал между двумя графическими процессорами. Чтобы избежать конкуренции между графическими процессорами, нечетные графические процессоры разгружают активации для нечетных слоев, в то время как четные графические процессоры разгружают активацию для четных слоев. Это имеет решающее значение для полного использования пропускной способности PCIe. Планирование разгрузки четных и нечетных слоев между графическими процессорами предотвращает конфликты по каналу PCIe, позволяя каждому графическому процессору в полной мере использовать пропускную способность PCIe, когда ему необходимо разгрузить

**Пояснение:** Когда одна GPU работает, вторая разгружает

## Massive scale sparce model inference

### Orchestration of Tensor, Data, &Expert Parallelism for MoE

Мы используем тензорный параллелизм, обозначенный на рисунке ниже как тензорный срез (для неэкспертных параметров) и экспертный срез (для экспертных параметров), чтобы разделить отдельные параметры между несколькими графическими процессорами и использовать совокупную пропускную способность памяти между графическими процессорами. Однако тензорный параллелизм может эффективно масштабироваться только до нескольких графических процессоров из-за накладных расходов на связь и мелкозернистого параллелизма. Чтобы решить эту проблему, мы используем экспертный параллелизм в сочетании с тензорным параллелизмом для масштабирования параметров экспертов до сотен графических процессоров. Экспертный параллелизм не снижает гранулярность вычислений отдельных операторов, что позволяет нашей системе использовать совокупную пропускную способность памяти на сотнях графических процессоров. Чтобы масштабировать неэкспертные вычисления до того же количества графических процессоров, мы используем параллелизм данных без накладных расходов на обмен данными.

![Orcestration](../Pictures/DeepSpeed-inference/Orcestration.png)

### PCC: Parallelism Coordinated Communication for MoE

Экспертный параллелизм размещает экспертных операторов между графическими процессорами и требует обмена данными «все ко всем» между всеми экспертно-параллельными графическими процессорами. Однако масштабировать экспертный параллелизм на сотни устройств, необходимых для вывода разреженной модели, неэффективно, поскольку задержка линейно увеличивается с увеличением количества устройств. К счастью, при объединении экспертного параллелизма и тензорного в рамках одной модели существуют возможности для оптимизации связи, которые могут уменьшить задержку связи. Обратите внимание, что тензорная срезка разделяет отдельные операторы между графическими процессорами и требует all-reduce между ними. Операция all-reduce в тензорной срезке реплицирует данные между задействованными устройствами. При выполнении тензорно-параллельных операторов, за которыми следуют экспертно-параллельные операторы, такая репликация позволяет создать оптимизированное расписание взаимодействия для оператора «все ко всем», не требующее взаимодействия между всеми экспертными параллельными процессами: «все ко всем» может происходить только в пределах подмножества устройств, которые имеют один и тот же ранг тензорного среза, поскольку данные по рангам тензорно-параллельного сечения реплицируются (рисунок ниже). В результате задержка подхода «все ко всем» ограничена $O(p/L)$, а не $O(p)$, где $L$ — степень тензорного параллелизма, а $p$ — общее количество устройств графического процессора.

Аналогичным образом, при выполнении операторов экспертного параллелизма, за которыми следуют операторы тензорного среза, окончательный метод «все ко всем» может быть выполнен таким же образом, но на этот раз с последующим оператором allgather между тензорно-параллельными рангами для репликации данных, необходимых для тензорного среза (рисунок ниде). Это снижает накладные расходы на задержку от $O(p)$ до $O(p/L)$ + $O(L)$.


![PCC](../Pictures/DeepSpeed-inference/PCC.png)

### Highly Optimized Computation Kernels for MoE

## Democratization of large model inference

Использование ZeRO-inference([ссылка](ZeRO.ipynb))

# DeepSpeed-FastGen: High-throughput Text Generation for LLMs via MII and DeepSpeed-Inference ([ссылка](https://arxiv.org/pdf/2401.08671))